In [20]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import quaternion as q
import os
from tqdm.notebook import tqdm
import joblib
os.environ['CUDA_VISIBLE_DEVICES'] = "6"
os.environ['GLOG_minloglevel'] = "3"
os.environ['MAGNUM_LOG'] = "quiet"
os.environ['HABITAT_SIM_LOG'] = "quiet"
import habitat
from habitat import get_config
from habitat.sims import make_sim
config = get_config()
cfg = config
cfg.defrost()
habitat_api_path = os.path.join(os.path.dirname(habitat.__file__), '../')
cfg.DATASET.SCENES_DIR = os.path.join(habitat_api_path, cfg.DATASET.SCENES_DIR)
cfg.DATASET.DATA_PATH = os.path.join(habitat_api_path, cfg.DATASET.DATA_PATH.replace("habitat-test-scenes","gibson"))
config.TASK_CONFIG = add_panoramic_camera(config.TASK_CONFIG, normalize_depth=True)
cfg.DATASET.TYPE = "PointNav-v1"
cfg.SIMULATOR.RGB_SENSOR.HEIGHT = 128
cfg.SIMULATOR.RGB_SENSOR.WIDTH = 128
cfg.SIMULATOR.DEPTH_SENSOR.HEIGHT = 128
cfg.SIMULATOR.DEPTH_SENSOR.WIDTH = 128
cfg.SIMULATOR.EQUIRECT_RGB_SENSOR.HEIGHT = 128
cfg.SIMULATOR.EQUIRECT_RGB_SENSOR.WIDTH = 256
cfg.SIMULATOR.EQUIRECT_RGB_SENSOR.UUID = 'pano_rgb'
cfg.SIMULATOR.EQUIRECT_DEPTH_SENSOR.HEIGHT = 128
cfg.SIMULATOR.EQUIRECT_DEPTH_SENSOR.WIDTH = 256
cfg.SIMULATOR.EQUIRECT_DEPTH_SENSOR.UUID = 'pano_depth'
cfg.TASK.SENSORS = cfg.SIMULATOR.AGENT_0.SENSORS = ['RGB_SENSOR', 'DEPTH_SENSOR', 'EQUIRECT_RGB_SENSOR', 'EQUIRECT_DEPTH_SENSOR']
cfg.freeze()

from habitat.datasets import make_dataset
dataset = make_dataset('PointNav-v1')
cfg.defrost()
cfg.DATASET.TYPE = 'PointNav-v1'
cfg.DATASET.SPLIT = 'train'
cfg.freeze()
train_scenes = dataset.get_scenes_to_load(cfg.DATASET)
cfg.defrost()
cfg.DATASET.SPLIT = 'val'
cfg.freeze()
val_scenes = dataset.get_scenes_to_load(cfg.DATASET)
all_scenes = train_scenes + val_scenes
print(f'Total {len(all_scenes)} scenes found')


2022-06-24 14:25:27,691 Initializing dataset PointNav-v1


Total 86 scenes found


In [27]:
data_dir = '/disk3/nuri/habitat_image_dataset'
os.makedirs(data_dir, exist_ok=True)

In [28]:
for scene in tqdm(all_scenes):
    try: sim.close()
    except: pass
    random_scene = np.random.choice(all_scenes)
    cfg.defrost()
    cfg.SIMULATOR.SCENE = os.path.join(cfg.DATASET.SCENES_DIR,'gibson_habitat/{}.glb'.format(random_scene))
    cfg.freeze()
    sim = make_sim(id_sim=cfg.SIMULATOR.TYPE, config=cfg.SIMULATOR)
    split = 'train' if scene in train_scenes else 'val'
    for t in tqdm(range(5000)):
        point = sim.sample_navigable_point()
        x,y,z = point
        x, y, z = round(x,2),round(y,2),round(z,2)
        if not sim.is_navigable(np.array([x,y,z])):
            continue
        angle = 2*np.pi*np.random.rand()
        angle = round(angle,2)
        random_rotation = q.from_euler_angles([0, angle, 0])
        obs = sim.get_observations_at(point, random_rotation)
        file_name = f'{scene}_{x:.1f}_{y:.1f}_{z:.1f}_{angle:.1f}.dat.gz'

        data_dict = obs
        data_dict.update({'position': np.array([x,y,z]), 'rotation': random_rotation.components})
        joblib.dump(data_dict, os.path.join(data_dir,split,file_name))
    sim.close()

  0%|          | 0/86 [00:00<?, ?it/s]

2022-06-24 14:28:38,497 initializing sim Sim-v0


  0%|          | 0/5000 [00:00<?, ?it/s]

2022-06-24 14:30:46,929 initializing sim Sim-v0


  0%|          | 0/5000 [00:00<?, ?it/s]

...

2022-06-24 14:47:10,898 initializing sim Sim-v0


  0%|          | 0/5000 [00:00<?, ?it/s]

2022-06-24 14:49:54,490 initializing sim Sim-v0


  0%|          | 0/5000 [00:00<?, ?it/s]

ValueError: write to closed file

NameError: name 'joblib' is not defined

'Adrian_-7.3_0.1_-5.8_1.2.dat.gz'